In [57]:
import io
import spacy
from collections import Counter
from collections import defaultdict 
import torch.nn as nn
import torchvision.models as models
import torchvision
import torch
from  torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer


In [14]:
IMAGES_PATH = './flickr8/Images'
CAPTIONS_PATH = './flickr8/captions.txt'

In [ ]:
!python -m spacy download en_core_web_sm

In [15]:
def load_img_caption_dict():
    img_capt_dict = defaultdict(list)
    with open(CAPTIONS_PATH, 'r') as captions_file:
        for line in captions_file.readlines():
            if line.startswith("image"):
                # header
                continue
            
            else:
                current_line = line.split(',')
                img = current_line[0]
                capt = current_line[1]
                img_capt_dict[img].append(capt)
    return img_capt_dict

In [16]:
tokenizer = en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
counter = Counter()
img_caption_dict = load_img_caption_dict()

In [20]:
img_caption_dict['1000268201_693b08cb0e.jpg']

['A child in a pink dress is climbing up a set of stairs in an entry way .\n',
 'A girl going into a wooden building .\n',
 'A little girl climbing into a wooden playhouse .\n',
 'A little girl climbing the stairs to her playhouse .\n',
 'A little girl in a pink dress going into a wooden cabin .\n']

In [22]:
counter = Counter()
for img_key in img_caption_dict:
    for caption in img_caption_dict[img_key]:
        counter.update(tokenizer(caption))
        

print(len(counter))

9020


In [43]:
caption_vocab = vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])
caption_vocab.lookup_token(100)
caption_vocab.set_default_index(caption_vocab["<unk>"])


In [55]:
# tokenizer(img_caption_dict[list(img_caption_dict.keys())[0]])
caption_vocab[tokenizer(img_caption_dict[list(img_caption_dict.keys())[0]][0])[0]]


data = []
for img_key in img_caption_dict:
    for caption in img_caption_dict[img_key]:
        tokens = tokenizer(caption)
        tensor = torch.tensor([caption_vocab[token] for token in tokens], dtype=torch.long)
        # TODO - load image data instead of using image key
        data.append((img_key, tensor))

data[0]

('1000268201_693b08cb0e.jpg',
 tensor([ 4,  5,  6,  7,  8,  9, 10, 11, 12,  7, 13, 14, 15,  6, 16, 17, 18, 19,
         20]))